# Data Loading

In [4]:
##################
# All imports 
##################

#sys libs
import os
import sys
import random
import warnings
warnings.filterwarnings("ignore")

#data manupulation libs
import pandas as pd
import numpy as np
#from sklearn.model_selection import train_test_split

#from pandarallel import pandarallel

# Initialization
#pandarallel.initialize()

#string manupulation libs
import re
import string
from string import digits
import spacy
import nltk
from nltk.corpus import stopwords 
nltk.download('stopwords')

#torch libs
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim

import transformers

from transformers import BertTokenizer, AutoTokenizer

# data manipulations
from pathlib import Path
import uuid
import pydicom

from PIL import Image

import cv2
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     /media/SharedUsers/dlc19/home/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Data Preparation

In [5]:
# Used to make the csv with raw reports, not really important anymore once we have the reports in a csv
cxr_root_path = "/media/Data/ComputerVision/mimic_cxr_jpg_small_chest_radiograph/data/physionet.org/files/mimic-cxr-jpg/2.0.0/"
df_record = pd.read_csv('{}/cxr-record-list.csv.gz'.format(cxr_root_path), sep=',')
df_split = pd.read_csv('{}/mimic-cxr-2.0.0-split.csv.gz'.format(cxr_root_path))

df_temp = df_split.merge(df_record, on=['subject_id', 'study_id', 'dicom_id'], how='left')

df_sections = pd.read_csv('{}/mimic-cxr-sections/mimic_cxr_sectioned.csv'.format(cxr_root_path))

# if you already have the df_raw_reports.csv just uncomment the following line and skipp to the vocabulary class

# extract the csv into a dataframe
df_raw_reports = pd.read_csv('df_raw_reports.csv')
# discard duplicates of the reports as there were originally more rows than reports (as there is sometimes more images per study)
df_raw_reports= df_raw_reports.drop_duplicates(subset = ["raw_report"])
#restart the index column as it was filtered from larger data and the indices were messed up
df_raw_reports.reset_index(inplace=True)
#discard some unimportant columns
df_raw_reports = df_raw_reports.drop('index', 1)
df_raw_reports = df_raw_reports.drop('Unnamed: 0', 1)
# print the head to check if looks like intended
df_raw_reports.head()


dicom_id  study_id  subject_id  split  \
0  02aa804e-bde0afdd-112c0b34-7bc16630-4e384014  50414267    10000032  train   
1  2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab  53189527    10000032  train   
2  68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714  53911762    10000032  train   
3  ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c  56699142    10000032  train   
4  096052b7-d256dc40-453a102b-fa7d01c6-1b22c6b4  57375967    10000764  train   

                                                path  \
0  files/p10/p10000032/s50414267/02aa804e-bde0afd...   
1  files/p10/p10000032/s53189527/2a2277a9-b0ded15...   
2  files/p10/p10000032/s53911762/68b5c4b1-227d048...   
3  files/p10/p10000032/s56699142/ea030e7a-2e3b134...   
4  files/p10/p10000764/s57375967/096052b7-d256dc4...   

                                          raw_report  
0                                   FINAL REPORT\...  
1                                   FINAL REPORT\...  
2                                   FINAL REPORT\...  
3                                   FINAL REPORT\...  
4                                   FINAL REPORT\...

In [6]:
# Checking test dataset number
df_split.head()
print(df_split.shape)

split = df_split["split"].tolist()
idx = 0
for elem in split:
    if elem == "test":
        idx+=1

print(idx)

(377110, 4)
5159


### Creating raw reports dataframe (Skip if done)

In [32]:
df_new_sections = df_sections.rename(columns={'study': 'study_id'})

In [33]:
# Adding the raw report to the sections df


paths = df_record["path"].tolist()
study_ids = df_record["study_id"].tolist()

study_ids_new=[]

for study in study_ids:
    
    new_id = 's' + str(study)
    study_ids_new.append(new_id)


In [9]:
# creates dataframe with study id and raw reports, prints the name of the main directory on enter so you can see the progress
# overall there is p10-p19 directories, should take around 25-30 min to run as far as I remember

import ntpath

def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

def searchDirectory(mainDirectory,df_raw_reports):


    directory = '/media/Data/ComputerVision/mimic_cxr_jpg_small_chest_radiograph/data/physionet.org/files/mimic-cxr-jpg/2.0.0/files/' + mainDirectory
    report_ext = '.txt'
    image_ext = '.jpg'
    
    print(mainDirectory)
    for subDirectory in os.listdir(directory):
        full_path = directory + '/' + subDirectory
       
        for root, dirs, files in os.walk(full_path):
            
            for filename in files:

                if filename.endswith(report_ext):
                    with open(os.path.join(root, filename), 'r') as report:
                        contents = report.read()
                    
                    df_temp = {"study_id": path_leaf(filename)[1:9], "raw_report": contents}
                    df_raw_reports = df_raw_reports.append(df_temp, ignore_index = True)
        
    return df_raw_reports



    
directories = ['p10','p11','p12','p13','p14','p15','p16','p17', 'p18', 'p19' ]


df_raw_reports = pd.DataFrame([], columns = ['study_id', 'raw_report'])

for directory in directories:
    df_raw_reports = searchDirectory(directory, df_raw_reports)


p10
p11


KeyboardInterrupt: 

In [13]:
df_raw_reports.head()
df_raw_reports.to_csv('raw_reports.csv', index=False)


df_raw_reports = pd.read_csv('df_raw_reports.csv')

df_raw_reports= df_raw_reports.drop_duplicates(subset = ["raw_report"])
df_raw_reports.reset_index(inplace=True)
df_raw_reports = df_raw_reports.drop('index', 1)
df_raw_reports = df_raw_reports.drop('Unnamed: 0', 1)
df_raw_reports.head()

dicom_id  study_id  subject_id  split  \
0  02aa804e-bde0afdd-112c0b34-7bc16630-4e384014  50414267    10000032  train   
1  2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab  53189527    10000032  train   
2  68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714  53911762    10000032  train   
3  ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c  56699142    10000032  train   
4  096052b7-d256dc40-453a102b-fa7d01c6-1b22c6b4  57375967    10000764  train   

                                                path  \
0  files/p10/p10000032/s50414267/02aa804e-bde0afd...   
1  files/p10/p10000032/s53189527/2a2277a9-b0ded15...   
2  files/p10/p10000032/s53911762/68b5c4b1-227d048...   
3  files/p10/p10000032/s56699142/ea030e7a-2e3b134...   
4  files/p10/p10000764/s57375967/096052b7-d256dc4...   

                                          raw_report  
0                                   FINAL REPORT\...  
1                                   FINAL REPORT\...  
2                                   FINAL REPORT\...  
3                                   FINAL REPORT\...  
4                                   FINAL REPORT\...

### Classes for the Data Loader
- Include: loading data, text preprocessing, words frequency check, tokenization, tokens-IDs-conversion 

In [7]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [12]:
##############################
# Vocabulary class
##############################

class Vocabulary:
  
    '''
    __init__ method is called by default as soon as an object of this class is initiated
    we use this method to initiate our vocab dictionaries
    '''
    def __init__(self, freq_threshold, max_size):
        '''
        freq_threshold : the minimum times a word must occur in corpus to be treated in vocab
        max_size : max source vocab size. Eg. if set to 10,000, we pick the top 10,000 most frequent words and discard others
        '''
        
        self.freq_threshold = freq_threshold
        self.max_size = max_size
        self.stop_words = set(stopwords.words('english'))
        self.tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
        
    '''
    a simple tokenizer to split on space and converts the sentence to list of words
    '''
    @staticmethod
    def preprocessing(text):
        cleanedReport = re.sub(r'[^\w\s]','',text)            # remove punctuation (not word characters and whitespace)
        cleanedReport = re.sub('_', '', cleanedReport)        # remove __ in the report
        cleanedReport = re.sub(r'[\d-]', '', cleanedReport)   # remove numbers in the report 
        cleanedReport = re.sub('\n', '', cleanedReport)

        return cleanedReport                                  # should be just a string
    
    '''
    vocabulary frequency check:
    '''
    def build_vocabulary(self, sentence_list):
        # calculate the frequencies of each word first to remove the words with freq < freq_threshold
        frequencies = {}  #init the freq dict
        #idx = 4 #index from which we want our dict to start. We already used 4 indexes for pad, start, end, unk
        idx = 4
        
        i=0
        for sentence in sentence_list:
            i+=1
            if (i % 10000 == 0):
                print(i) 
                
            preprocessed = self.preprocessing(sentence)
            preprocessedList = list(preprocessed.split(" "))
            for word in preprocessedList:
                if word not in frequencies.keys():
                    frequencies[word]=1
                else:
                    frequencies[word]+=1
              
                    
        #limit vocab by removing low freq words
        frequencies = {k:v for k,v in frequencies.items() if v > self.freq_threshold} 
        
        #limit vocab to the max_size specified
        frequencies = dict(sorted(frequencies.items(), key = lambda x: -x[1])[:self.max_size-idx]) # idx =4 for pad, start, end , unk
            
        return frequencies

            
    '''
    tokenization and converting tokens to IDs
    '''    
    def tokenization(self, text):
        
        # change here to use different tokenizer models:

        # add tokenizer
        tokenized_text = self.tokenizer.tokenize(text)
        tokenized_text = [word for word in tokenized_text if word not in self.stop_words]    # Stopword removal
        
        tokenized_report = ["[CLS]"]
        tokenized_report += tokenized_text
        tokenized_report.append("[SEP]")
        numericalized_report = tokenizer.convert_tokens_to_ids(tokenized_report)
        
#         # Another encoding method which works on bert-base-uncased tokenizer but not working well with not the others...
#         text_str = ' '.join([str(elem) for elem in text])
#         encoded = tokenizer.encode_plus(text=text_str,  # the sentence to be encoded
#                                         add_special_tokens=True,  # Add [CLS] and [SEP]
#                                         return_attention_mask = True,  # Generate the attention mask
#                                         #return_tensors = 'pt',  # ask the function to return PyTorch tensors
#                                         )
        
        return numericalized_report

In [13]:
###############
# Train dataset
###############

class TrainDataset(Dataset):
    
    def __init__(self, df_train, target_column, transform=None, freq_threshold = 0,
                vocab_max_size = 5000):
       
        self.df_train = df_train
        
        self.transform = transform
        
        self.target_text = df_train[target_column]

        self.report_vocab = Vocabulary(freq_threshold, vocab_max_size)
        self.frequencyDict = self.report_vocab.build_vocabulary(self.target_text.tolist())  # build vocab for whole thing (list of whole thing)
        
    def __len__(self):
        return len(self.df_train)
    

    def __getitem__(self, index):
  
        target_text = self.target_text[index]                             # string
        target_text = self.report_vocab.preprocessing(target_text)
        
        for word in target_text.split():
            if word not in self.frequencyDict.keys():
                target_text = target_text.replace(word, "")
            
        numericalized_report = self.report_vocab.tokenization(target_text)
        
        # uncomment the following to convert the list to tensor and return (apparently BERT works without tensor)
        # return torch.tensor(numericalized_report)
        return numericalized_report

#### Example:

In [14]:
# Here's a little example on just 10 reports
#df_train_test = df_raw_reports[0:10]
df_train_test = df_raw_reports[0:10]

train_dataset = TrainDataset(df_train_test, "raw_report")

In [15]:
# we can see the results, e.g.
example = train_dataset[0]
print(example)
print("\n")
print("Number of tokens: ", len(example))

[101, 1509, 2592, 8179, 2229, 185, 1161, 2495, 1204, 12754, 175, 1207, 15415, 14375, 1116, 174, 7501, 8974, 5531, 2229, 185, 1161, 11937, 7577, 3839, 9505, 17811, 20994, 185, 1513, 12602, 174, 3101, 17268, 185, 1673, 1818, 12858, 25632, 20557, 6873, 5552, 11769, 7409, 4233, 2620, 4248, 15070, 6719, 3621, 2660, 16418, 2050, 14196, 27316, 2999, 16973, 1933, 1286, 13093, 9046, 1439, 7209, 3077, 1181, 3105, 14701, 8362, 16996, 23822, 1895, 13306, 19353, 24211, 1785, 16530, 1286, 3971, 5001, 10346, 2382, 8351, 12104, 3621, 2660, 16091, 13505, 7637, 1616, 1965, 102]


Number of tokens:  91


In [16]:
# Conversion back to tokens:
print("Tokens conversion:")
print(tokenizer.convert_ids_to_tokens(example))
print("\n")

# Decode it back into sentences (tokens of subwords will be merged together)
print("Decoding:")
print(tokenizer.decode(example))

Tokens conversion:
['[CLS]', 'final', 'report', 'examination', 'chest', 'p', '##a', 'la', '##t', 'indication', 'f', 'new', 'onset', '##cite', '##s', 'e', '##val', 'infection', 'technique', 'chest', 'p', '##a', 'lateral', 'comparison', 'none', 'findings', 'focal', 'consolidation', 'p', '##le', '##ural', 'e', '##ff', '##usion', 'p', '##ne', '##um', '##oth', '##orax', 'bilateral', 'nod', '##ular', 'op', '##ac', '##ities', 'likely', 'represent', 'nipple', 'shadows', 'card', '##io', '##media', '##st', '##inal', 'silhouette', 'normal', 'clips', 'project', 'left', 'lung', 'potentially', 'within', 'breast', 'image', '##d', 'upper', 'abdomen', 'un', '##rem', '##ark', '##able', 'chronic', 'def', '##orm', '##ity', 'posterior', 'left', 'sixth', 'seventh', 'ribs', 'noted', 'impression', 'acute', 'card', '##io', '##pu', '##lm', '##ona', '##ry', 'process', '[SEP]']


Decoding:
[CLS] final report examination chest pa lat indication f new onsetcites eval infection technique chest pa lateral comparison 

### Chunking to unify token lengths of each sample 

In [17]:
####################
# Chunking Function 
####################

def chunking(dataset, df, chunk_size = 128):
    
    concat = []
    for i in range(len(dataset)):     # Add all tokens together in one list
        concat.extend(dataset[df.index[i]])
        
    total_length = len(concat)
    print("Total concatenated length: ", total_length)
    
    chunks = lambda concat, chunk_size: [concat[i:i+chunk_size] for i in range(0, total_length, chunk_size)]  
    chunked_text = chunks(concat, chunk_size)
    
    if len(chunked_text[-1]) < chunk_size:
        print("Removed last chunk: ", len(chunked_text[-1]), " tokens")
        chunked_text.pop()     # remove last element 
    
    return chunked_text

##### (Example - for 3 reports):

In [18]:
example_df = df_raw_reports.sample(n=3)
example_dataset = TrainDataset(example_df, "raw_report")
chunked_text = chunking(example_dataset, example_df)

for chunk in chunked_text:
    print("\n")
    print(tokenizer.decode(chunk))

Total concatenated length:  323
Removed last chunk:  67  tokens


[CLS] final report indication year old man bradycardiap temporary wire placement temporary wire location pulmonary edema technique portable chest radiograph comparison chest radiograph dated findings ett cm carina leftj centralnous catheter removed enteric tube extends stomach terminates beyond view radiograph right femoral temporary wire terminates likely right ventricle lateral view could provide definitive information round opacity irregular border right upper lobe concerning pneumonia persistent left lower lobe atelectasis hila pulmonary vasculatures normal cardiomediastinal silhouette normal pleural effusion pneumothorax fractures impression right upper lobe opacity concerning pneumonia


pulmonary edema temporary wire positioned tip right ventricleification findings discussedrd telephone pm minutes discovery findings [SEP] [CLS] final report history yearold male chf history possible gout presents chest pain days co

### Chunking all training reports (Skip if done!) 

In [77]:
# # Uncomment if you want to do it yourself!
# # For all training reports:
# trainDf = df_raw_reports[0:len(df_raw_reports.index)]
# train_dataset = TrainDataset(trainDf, "raw_report")
# chunked_train_dataset = chunking(train_dataset, trainDf)    # chunk size = 128 

# # Save as .csv file:
# chunked_trainDF = pd.DataFrame(chunked_train_dataset)
# chunked_trainDF.to_csv("chunked_trainDatasetAll.csv")

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
Total concatenated length:  25994905
Removed last chunk:  25  tokens


Each row contains one training sample with 128 tokens, in total there are 204085 training samples:

### Reading the chunked train dataset

In [21]:
chunked_trainDataset = pd.read_csv('chunked_trainDatasetAllNoStopWord.csv')
chunked_trainDataset = chunked_trainDataset.drop('Unnamed: 0', 1)
print("Number of chunked samples:    ", chunked_trainDataset.shape[0])
print("Number of tokens per samples: ", chunked_trainDataset.shape[1])
chunked_trainDataset.head()

Number of chunked samples:     154696
Number of tokens per samples:  128


0      1     2      3      4      5      6      7      8      9  ...  \
0    101   1509  2592   8179   2229    185   1161   2495   1204  12754  ...   
1   2999   8682  2330    185   1513  12602    174   3101  17268    185  ...   
2   5351  10170  1700   7577   2070  21217   9505   8682   2330  17811  ...   
3  17811  20994  1286  13093   2259   3566   4311  22631  20673   2129  ...   
4    185   1673  1818  12858  25632    175  14687  26600   5048  14494  ...   

     118    119    120    121    122    123    124   125    126    127  
0   5815  14255  18834   1116   2999  26600    191  2225  21608   5332  
1   3161   1197  15342   1548   5531  22172   2229  2070  21217   3836  
2  11019   1233   6617  11531   1116   1762   1499  2999   2060   8351  
3   1849   2554  17811  20994    185   1513  12602   174   3101  17268  
4   5815  14255  18834   1116  16684   1353    185  1513  12602    174  

[5 rows x 128 columns]

##### Example in converting 3 samples (first 3 rows) back to words:

In [22]:
chunked_dfList = chunked_trainDataset.values.tolist()   # It is a big list containing 203085 samples (list)
print("Length of the list:  ", len(chunked_dfList))
print(type(chunked_dfList[0]))
print("\n")

for i in range(3):
    print(chunked_dfList[i])
    print(tokenizer.convert_ids_to_tokens(chunked_dfList[i]))
    print("\n")

Length of the list:   154696
<class 'list'>


[101, 1509, 2592, 8179, 2229, 185, 1161, 2495, 1204, 12754, 175, 1207, 15415, 14375, 1116, 174, 7501, 8974, 5531, 2229, 185, 1161, 11937, 7577, 3839, 9505, 17811, 20994, 185, 1513, 12602, 174, 3101, 17268, 185, 1673, 1818, 12858, 25632, 20557, 6873, 5552, 11769, 7409, 4233, 2620, 4248, 15070, 6719, 3621, 2660, 16418, 2050, 14196, 27316, 2999, 16973, 1933, 1286, 13093, 9046, 1439, 7209, 3077, 1181, 3105, 14701, 8362, 16996, 23822, 1895, 13306, 19353, 24211, 1785, 16530, 1286, 3971, 5001, 10346, 2382, 8351, 12104, 3621, 2660, 16091, 13505, 7637, 1616, 1965, 102, 101, 1509, 2592, 8179, 2229, 185, 1161, 2495, 1204, 12754, 1607, 175, 1603, 1757, 2184, 5531, 2229, 185, 1161, 11937, 7577, 9505, 17688, 2394, 2050, 14196, 20844, 5815, 14255, 18834, 1116, 2999, 26600, 191, 2225, 21608, 5332]
['[CLS]', 'final', 'report', 'examination', 'chest', 'p', '##a', 'la', '##t', 'indication', 'f', 'new', 'onset', '##cite', '##s', 'e', '##val', 'infection', 'tec